# Amazon Titan を使ったテキスト要約

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*

## 概要
大規模なドキュメントを扱う際、入力テキストがモデルのコンテキスト長に収まらない、モデルが大規模ドキュメントで幻覚 (ハルシネーション) を起こす、メモリ不足エラーが発生するなどの課題に直面することがあります。

これらの問題を解決するため、プロンプトのチャンキング (分割) とチェーン (結合) のアーキテクチャを使用します。このアーキテクチャは、言語モデルによって動作するアプリケーションの開発に広く利用されている [LangChain](https://python.langchain.com/docs/get_started/introduction.html) フレームワークを活用しています。

### アーキテクチャ

![](./images/42-text-summarization-2.png)

アーキテクチャ:

1. 大規模なドキュメント (または小さなドキュメントを結合した巨大なファイル) を読み込みます
1. LangChain ユーティリティを使用して、複数の小さなチャンクに分割します (チャンキング)
1. 最初のチャンクをモデルに送信し、モデルは対応する要約を返します
1. LangChain は次のチャンクを取得し、返された要約に追加して、結合したテキストを新しいリクエストとしてモデルに送信します。このプロセスはすべてのチャンクが処理されるまで繰り返されます
1. 最後に、全コンテンツに基づいた最終的な要約が得られます

### ユースケース
このアプローチは、通話記録、会議記録、書籍、記事、ブログ記事、その他の関連コンテンツを要約するのに使用できます。

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## 長文の要約

### Boto3 を使った LangChain の構成

LangChain では、boto3 セッション情報を LangChain に渡すことで、Bedrock にアクセスできます。Boto3 セッション情報として None を渡すと、LangChainは環境からセッション情報を取得しようとします。

正しいクライアントが使用されることを確認するために、ユーティリティメソッドを利用してインスタンス化します。

LangChain の Bedrock クラスに LLM を指定する必要があり、推論の引数を渡すことができます。ここでは `model_id` に Amazon Titan Text Express モデルを指定し、モデルの推論パラメータを `textGenerationConfig` に渡しています。

In [ ]:
from langchain_community.llms.bedrock import Bedrock
modelId = "amazon.titan-text-express-v1" # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
llm = Bedrock(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=boto3_bedrock,
)

### 多くのトークンを持つテキストファイルの読み込み

`letters` ディレクトリには、Amazon の CEO が 2022 年の株主へ送った手紙 [Amazon's CEO letter to shareholders in 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) のテキストファイルが保存されています。  
次のセルはテキストファイルを読み込み、ファイル内のトークン数を数えます。  

このテキストファイルのトークン数が、このモデルの最大トークン数を超えていることを示す警告が表示されるでしょう。

In [ ]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

### 長文をチャンクに分割

このテキストはプロンプトに収めるには長すぎるので、小さなチャンクに分割します。

LangChain の `RecursiveCharacterTextSplitter` は、各チャンクのサイズが `chunk_size` より小さくなるまで、長いテキストを再帰的にチャンクに分割することをサポートしています。テキストは `separators=["\n\n", "\n"]` でチャンクに分割されるため、各段落が複数のチャンクに分割されるのを回避できます。

1 チャンク当たり 6000 文字を使用することで、それぞれの部分に対して要約を得ることができます。チャンク内のトークン数またはワードピースの数は、テキストによって異なります。

In [ ]:
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

### チャンクを要約し結合する

最初のドキュメントのトークン数を確認しましたが、他のドキュメントでもトークン数が同程度であると仮定し作業を継続します。
LangChain の [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html) をテキスト要約に使用します。 

`load_summarize_chain` は、要約のための 3 つの方法を提供しています。
- `stuff` - すべてのチャンクを 1 つのプロンプトに入れます。したがって、これはトークンの最大制限に達してしまいます。
- `map_reduce` - 各チャンクを要約し、要約を結合して、結合した要約を要約します。結合した要約が大きすぎる場合、エラーが発生します。
- `refine` - 最初のチャンクを要約し、次に2番目のチャンクを最初の要約とともに要約します。同じプロセスを、すべてのチャンクが要約されるまで繰り返します。

`map_reduce` と `refine` は LLM を複数回呼び出すため、最終的な要約を取得するのに時間がかかります。

ここでは `map_reduce` を利用します。

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

> ⏰ **注:** ドキュメントの数、Bedrock のリクエストレートクォータ、設定されたリトライ設定によっては、以下のチェーンの実行に時間がかかる場合があります。

In [ ]:
output = ""
try:
    
    output = summary_chain.run(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
print_ww(output.strip())